In [2]:
import numpy as np
from scipy import stats


Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ2) лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [5]:
print('Ans: ', round(stats.norm.ppf(1-0.003/2),4))

Ans:  2.9677


In [16]:
aspirin = np.array([1 if i < 104 else 0 for i in range(11037)])
placebo = np.array([1 if i < 189 else 0 for i in range(11034)])


В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [10]:
prob_aspirin = sum(aspirin)/len(aspirin)
prob_placebo = sum(placebo)/len(placebo)

print('Ans:', round(prob_placebo - prob_aspirin, 4))

Ans: 0.0077


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.  

In [11]:
def proportions_confint_diff_ind(sample1, sample2, alpha=0.05):
    z = scipy.stats.norm.ppf(1-alpha/2.)
    p1 = float(sum(sample1))/len(sample1)
    p2 = float(sum(sample2))/len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [12]:
print('Conf interval: {}'.format(proportions_confint_diff_ind(placebo, aspirin)))

Conf interval: (0.004687750675049439, 0.010724297276960124)



Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p1−p, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [13]:
def odds(p):
    return p/(1-p)

odds_asp = odds(prob_aspirin)
odds_plac = odds(prob_placebo)
print('Ans: ', round(odds_plac/odds_asp, 4))

Ans:  1.8321



Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;

установите random seed=0;

сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [14]:
def odds_data(data):
    p = sum(data)/len(data)
    return p/(1-p)

def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), size=(n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_interval(stat, alpha):
    bounds = np.percentile(stat, [100 * alpha/2., 100*(1 - alpha/2.)])
    return bounds

In [38]:
np.random.seed(0)

odds_aspirin_data = np.array(list(map(odds_data, get_bootstrap_samples(aspirin, 1000))))
odds_placebo_data = np.array(list(map(odds_data, get_bootstrap_samples(placebo, 1000))))


In [39]:
print('Ans: ', stat_interval(odds_placebo_data/odds_aspirin_data, 0.05))

Ans:  [1.44419465 2.34321168]
